In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import KFold
import seaborn as sns

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
train_dir = 'chest-xray/train'
test_dir = 'chest-xray/test'

In [2]:
# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Datasets
full_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
test_dataset = datasets.ImageFolder(test_dir, transform=test_transform)

In [3]:
# K-Fold setup
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)
batch_size = 32

In [4]:
# Function to create the model
def create_model():
    model = models.densenet121(pretrained=True)
    model.classifier = nn.Linear(model.classifier.in_features, 2)
    return model.to(device)

# Store best model state dict and fold metrics
best_val_acc = 0.0
best_model_state = None
history_all_folds = []

In [ ]:
# K-Fold Cross Validation
for fold, (train_ids, val_ids) in enumerate(kfold.split(full_dataset)):
    print(f"\n Fold {fold + 1}/{k_folds}")
    train_sub = Subset(full_dataset, train_ids)
    val_sub = Subset(full_dataset, val_ids)
    train_loader = DataLoader(train_sub, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_sub, batch_size=batch_size, shuffle=False)

    model = create_model()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train

        # Validation
        model.eval()
        val_loss_total = 0.0
        val_preds, val_labels = [], []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss_total += loss.item()
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_loss = val_loss_total / len(val_loader)
        val_acc = accuracy_score(val_labels, val_preds) * 100

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)

        print(f"Epoch {epoch+1}/10 | Train Loss: {train_loss:.4f}, Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%")

        # Save best model state
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()

    history_all_folds.append(history)


 Fold 1/5


/Users/mayuri/PycharmProjects/Lab0/pythonProject/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/mayuri/PycharmProjects/Lab0/pythonProject/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 | Train Loss: 0.1357, Acc: 95.09% | Val Loss: 0.0789, Acc: 97.13%
Epoch 2/10 | Train Loss: 0.0801, Acc: 97.10% | Val Loss: 0.0601, Acc: 97.61%
Epoch 3/10 | Train Loss: 0.0611, Acc: 97.77% | Val Loss: 0.0525, Acc: 97.99%
Epoch 4/10 | Train Loss: 0.0559, Acc: 97.82% | Val Loss: 0.0350, Acc: 98.56%


In [ ]:
#  Plot Loss and Accuracy
plt.figure(figsize=(14, 6))
for fold, hist in enumerate(history_all_folds):
    plt.subplot(1, 2, 1)
    plt.plot(hist['train_loss'], label=f'Train Fold {fold+1}')
    plt.plot(hist['val_loss'], linestyle='--', label=f'Val Fold {fold+1}')
    plt.title("Loss per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

    plt.subplot(1, 2, 2)
    plt.plot(hist['train_acc'], label=f'Train Fold {fold+1}')
    plt.plot(hist['val_acc'], linestyle='--', label=f'Val Fold {fold+1}')
    plt.title("Accuracy per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy (%)")

plt.subplot(1, 2, 1)
plt.legend()
plt.subplot(1, 2, 2)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Evaluation on Test Set using Best Model
print("\n Evaluating Best Model on Test Set...")
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
model = create_model()
model.load_state_dict(best_model_state)
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

In [ ]:
from sklearn.metrics import f1_score

# Metrics
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
print(f"\n Test Accuracy: {acc * 100:.2f}%")
print(f" Test F1 Score: {f1:.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['NORMAL', 'PNEUMONIA']))

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['NORMAL', 'PNEUMONIA'], yticklabels=['NORMAL', 'PNEUMONIA'])
plt.title("Confusion Matrix on Test Set")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.tight_layout()
plt.show()